In [115]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# Import the data set

In [116]:
lab_data = pd.read_csv('../../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data Cleaning

In [11]:
lab_data = pd.read_csv('../data/83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')

# Data cleaning
erroneous = lab_data[lab_data['Thorax_length'] == '.']
erroneous

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
253,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,6,3,female,.,1.65,0.573,1.415,1.988,1.986,0.898,1.207,1.003,.


In [12]:
similar_rows = lab_data[
    (lab_data['Species'] == 'D._aldrichi')
    & (lab_data['Population'] == 'Gogango_Creek')
    & (lab_data['Sex'] == 'female')
    & (lab_data['Thorax_length'] != ".")
]
similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])
similar_rows.head()

/var/folders/j_/9f1md2vd0_s6kpn12szf1t2c0000gn/T/ipykernel_67286/3776846164.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_rows['Thorax_length'] = pd.to_numeric(similar_rows['Thorax_length'])


,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
162,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,1,female,1.166,1.910,0.578,1.675,2.254,2.252,0.982,1.381,1.123,1.932
164,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,2,female,1.174,1.829,0.622,1.549,2.170,2.170,0.997,1.360,1.117,1.848
166,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,1,3,female,1.199,1.864,0.585,1.659,2.244,2.242,1.009,1.376,1.142,1.870
167,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,1,female,1.186,1.872,0.628,1.571,2.199,2.199,1.002,1.356,1.143,1.854
169,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,20,2,2,female,1.176,1.872,0.630,1.579,2.207,2.207,0.966,1.345,1.103,1.876


In [13]:
similar_row_median = similar_rows['Thorax_length'].median()

lab_data['Thorax_length'] = lab_data.apply(
    lambda x: (x['Thorax_length'] if x['Thorax_length'] != "." else similar_row_median),
    axis=1
)
lab_data.head()

lab_data['wing_loading'] = lab_data.apply(
    lambda x: (x['wing_loading'] if x['wing_loading'] != "." else round((x['l3'] / x['Thorax_length']), 3)),
    axis=1
)

print(len(similar_rows))
print(similar_row_median)

lab_data.loc[253]

87
1.141


Species            D._aldrichi
Population       Gogango_Creek
Latitude                -23.77
Longitude               150.17
Year_start                1994
Year_end                  1994
Temperature                 25
Vial                         6
Replicate                    3
Sex                     female
Thorax_length            1.141
l2                        1.65
l3p                      0.573
l3d                      1.415
lpd                      1.988
l3                       1.986
w1                       0.898
w2                       1.207
w3                       1.003
wing_loading             1.741
Name: 253, dtype: object

# Model Building

In [117]:
lab_data['Thorax_length'] = pd.to_numeric(lab_data['Thorax_length'])
lab_data['wing_loading'] = pd.to_numeric(lab_data['wing_loading'])

In [118]:
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,Thorax_length,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading
0,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,female,1.238,2.017,0.659,1.711,2.370,2.370,1.032,1.441,1.192,1.914
1,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,1,male,1.113,1.811,0.609,1.539,2.148,2.146,0.938,1.299,1.066,1.928
2,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,female,1.215,1.985,0.648,1.671,2.319,2.319,0.991,1.396,1.142,1.908
3,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,1,2,male,1.123,1.713,0.596,1.495,2.091,2.088,0.958,1.286,1.062,1.860
4,D._aldrichi,Binjour,-25.52,151.45,1994,1994,20,2,1,female,1.218,1.938,0.641,1.658,2.298,2.298,1.010,1.418,1.148,1.886


In [119]:
lab_data = shuffle(lab_data)
lab_data['class'] = lab_data.apply(
    lambda x: f"{x['Species']}, {x['Population']}",
    axis=1
)
lab_data.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l2,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class
1355,D._buzzatii,Grandchester,-27.68,152.45,1994,1994,30,8,1,female,...,1.690,0.533,1.527,2.060,2.060,0.899,1.259,0.984,1.839,"D._buzzatii, Grandchester"
704,D._aldrichi,Wahruna,-25.20,151.17,1994,1994,20,6,3,female,...,1.959,0.641,1.675,2.316,2.315,1.037,1.430,1.194,1.926,"D._aldrichi, Wahruna"
269,D._aldrichi,Gogango_Creek,-23.77,150.17,1994,1994,25,9,2,female,...,1.765,0.587,1.542,2.129,2.128,0.971,1.286,1.109,1.844,"D._aldrichi, Gogango_Creek"
1550,D._buzzatii,Oxford_Downs,-21.77,148.85,1994,1994,30,10,2,male,...,1.479,0.496,1.293,1.789,1.788,0.806,1.067,0.936,1.757,"D._buzzatii, Oxford_Downs"
1045,D._buzzatii,Gogango_Creek,-23.77,150.17,1994,1994,20,5,2,female,...,1.995,0.678,1.630,2.309,2.308,1.015,1.420,1.170,1.833,"D._buzzatii, Gogango_Creek"


In [120]:

train, test = train_test_split(lab_data, test_size=0.3)

In [121]:
number_columns = train.select_dtypes(include=['int64', 'float64']).columns

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(train[number_columns])

train[number_columns] = scaler.transform(train[number_columns])
test[number_columns] = scaler.transform(test[number_columns])

In [122]:
print(number_columns)
train.info()

Index(['Latitude', 'Longitude', 'Year_start', 'Year_end', 'Temperature',
       'Vial', 'Replicate', 'Thorax_length', 'l2', 'l3p', 'l3d', 'lpd', 'l3',
       'w1', 'w2', 'w3', 'wing_loading'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 1211 entries, 90 to 906
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Species        1211 non-null   object 
 1   Population     1211 non-null   object 
 2   Latitude       1211 non-null   float64
 3   Longitude      1211 non-null   float64
 4   Year_start     1211 non-null   float64
 5   Year_end       1211 non-null   float64
 6   Temperature    1211 non-null   float64
 7   Vial           1211 non-null   float64
 8   Replicate      1211 non-null   float64
 9   Sex            1211 non-null   object 
 10  Thorax_length  1211 non-null   float64
 11  l2             1211 non-null   float64
 12  l3p            1211 non-null   float64
 13  l3d            121

In [123]:
classifier = LogisticRegression(multi_class='multinomial')

classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(multi_class='multinomial')

In [124]:
train['class_predictions'] = classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [125]:
train.head()

,Species,Population,Latitude,Longitude,Year_start,Year_end,Temperature,Vial,Replicate,Sex,...,l3p,l3d,lpd,l3,w1,w2,w3,wing_loading,class,class_predictions
90,D._aldrichi,Binjour,0.365482,0.722222,0.0,0.0,0.5,0.777778,0.5,female,...,0.791436,0.879449,0.870608,0.870554,0.848708,0.846103,0.810289,0.880612,"D._aldrichi, Binjour","D._buzzatii, Oxford_Downs"
165,D._aldrichi,Gogango_Creek,0.661591,0.366667,0.0,0.0,0.0,0.000000,0.5,male,...,0.821823,0.863949,0.868127,0.867659,0.864391,0.826288,0.848875,0.953133,"D._aldrichi, Gogango_Creek","D._buzzatii, Grandchester"
211,D._aldrichi,Gogango_Creek,0.661591,0.366667,0.0,0.0,0.0,0.888889,0.5,female,...,0.904696,0.948335,0.954113,0.954094,0.971402,0.941215,0.981511,0.947706,"D._aldrichi, Gogango_Creek","D._buzzatii, Grandchester"
647,D._aldrichi,Oxford_Downs,1.000000,0.000000,0.0,0.0,1.0,0.444444,1.0,female,...,0.781768,0.838691,0.837536,0.837469,0.840406,0.838177,0.817524,0.882585,"D._aldrichi, Oxford_Downs","D._aldrichi, Binjour"
1404,D._buzzatii,Oxford_Downs,1.000000,0.000000,0.0,0.0,0.0,0.555556,0.0,male,...,0.874309,0.856487,0.878462,0.878412,0.882841,0.871202,0.869775,0.911692,"D._buzzatii, Oxford_Downs","D._buzzatii, Grandchester"


In [126]:
#Correct predictions
train.apply(
    lambda x: 1 if x['class_predictions'] == x['class'] else 0,
    axis=1
).sum()

191

In [127]:
#Incorrect predictions
train.apply(
    lambda x: 1 if x['class_predictions'] != x['class'] else 0,
    axis=1
).sum()

1020

In [128]:
# Retry with different parameters and regularization


In [129]:
regularised_classifier = LogisticRegression(C=1e5, solver='newton-cg', multi_class='multinomial')

regularised_classifier.fit(
    train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']],
    train[['class']]
)

/Users/brookqueree/Library/Caches/pypoetry/virtualenvs/comp3702_week3_practical-O5UAOgKJ-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100000.0, multi_class='multinomial', solver='newton-cg')

In [130]:
train['regularised_class_predictions'] = regularised_classifier.predict(train[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])

In [131]:
#Correct deep predictions
train.apply(
    lambda x: 1 if x['regularised_class_predictions'] == x['class'] else 0,
    axis=1
).sum()

273

In [132]:
test['softmax_predictions'] = classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])
test['regularised_softmax_predictions'] = regularised_classifier.predict(test[['Thorax_length', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'l2', 'l3p', 'wing_loading']])


correct_softmax_classification_test = test.apply(
    lambda x: 1 if x['softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Softmax correct predictions test set:" + str(correct_softmax_classification_test))

correct_regularised_classification_test = test.apply(
    lambda x: 1 if x['regularised_softmax_predictions'] == x['class'] else 0,
    axis=1
).sum()

print("Regularised softmax predictions test set:" + str(correct_regularised_classification_test))


Softmax correct predictions test set:80
Regularised softmax predictions test set:106


In [133]:
print(f"accuracy: {correct_regularised_classification_test / len(test)}")

accuracy: 0.20423892100192678
